<a href="https://colab.research.google.com/github/pavansai26/deployment-using-streamlit-library/blob/master/bank_note_authentication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.covariance import EllipticEnvelope

In [ ]:
df= pd.read_csv('/gdrive/My Drive/datasets_84803_196262_BankNote_Authentication.csv')

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.keys()

In [ ]:
df.columns

In [ ]:
df.count()

In [ ]:
df.isnull().sum()

**dimensionality reduction**

In [ ]:
import umap
reducer=umap.UMAP()

In [ ]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [ ]:

embedding=reducer.fit_transform(x)

In [ ]:
embedding.shape

In [ ]:
x.head()

In [ ]:
plt.scatter(embedding[:,0],embedding[:,1])
plt.gca().set_aspect('equal','datalim')
plt.title('UMAP PROJECTION OF THE  DATA', fontsize=24)

In [ ]:
y.head()

In [ ]:
qv1=df.iloc[:,0].quantile(0.25) # get quantile values and  iqr for outlier limits
qv1

In [ ]:
qv2=df.iloc[:,0].quantile(0.5)
qv2

In [ ]:
qv3=df.iloc[:,0].quantile(0.75)
qv3

In [ ]:
qv_limit=1.5*(qv3-qv1)
qv_limit

In [ ]:

un_outliers_mask=(df.iloc[:,0]>qv3+qv_limit) | (df.iloc[:,0]<qv1-qv_limit)
print(un_outliers_mask[:20])

In [ ]:

un_outliers_data=df.iloc[:,0][un_outliers_mask]
print(un_outliers_data[:50])

In [ ]:
import pylab
fig=pylab.figure(figsize=(6,6))
ax=fig.add_subplot(1,1,1)
for name,y in zip(un_outliers_mask,un_outliers_data):
  ax.text(1,y,name)
ax.boxplot(df.iloc[:,0])
#ax.set_ylabel(df.iloc[:,0])


bivariate outliers: create a scatterplot with an ellipse as a boundary for outliers.

In [ ]:
params=[df.iloc[:,0],df.iloc[:,1]]
params_data=np.array([df.iloc[:,0],df.iloc[:,1]]).T


compute the elliptical envelope

In [ ]:
ee=EllipticEnvelope()
ee.fit(params_data)

In [ ]:
biv_outliers_mask=ee.predict(params_data)== -1

In [ ]:
(biv_outliers_mask)

In [ ]:
biv_outliers_data=params_data[biv_outliers_mask]

In [ ]:
biv_outliers_data

In [ ]:
xx,yy=np.meshgrid(np.linspace(params_data[:,0].min(),params_data[:,0].max(),100),np.linspace(params_data[:,1].min(),params_data[:,1].max(),100))

In [ ]:
zz=ee.decision_function(np.c_[xx.ravel(),yy.ravel()])
zz=zz.reshape(xx.shape)

In [ ]:
#draw the scatterplot with the elliptical envelope and label the outliers

In [ ]:
fig=pylab.figure(figsize=(10,10))
ax=fig.add_subplot(1,1,1)
for name,xy in zip(biv_outliers_mask[:50],biv_outliers_data[:50]):
  ax.text(xy[0],xy[1],name)
ax.contour(xx,yy,zz,levels=[0],linewidth=2)
ax.scatter(params_data[:,0],params_data[:,1],color='red')

In [ ]:
#ellipse is obtained with the data outside the ellipse data points called outliers. but ellipse is not visible.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(x_train,y_train)

In [ ]:
y_pred= classifier.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test,y_pred)

In [ ]:
score

In [ ]:
classifier.predict([[2,3,4,1]])

In [ ]:
import pickle
pickle_out=open('classifier.pkl','wb')
pickle.dump(classifier,pickle_out)
pickle_out.close()

In [ ]:
pip install streamlit

In [ ]:
import streamlit as st

In [ ]:
pickle_in = open("classifier.pkl","rb")
classifier=pickle.load(pickle_in)

In [ ]:
def predict_note_authentication(variance,skewness,curtosis,entropy):
  prediction=classifier.predict([[variance,skewness,curtosis,entropy]])
  print(prediction)
  return prediction

In [ ]:
def main():
  st.title('bank authenticator')
  html_temp='''
  <div style= "background-color:tomato;padding:10px">
  <h2 style="color:white;text-align:center;">Streamlit Bank Authenticator ML App </h2>
  </div>'''
  st.markdown(html_temp,unsafe_allow_html=True)
  variance = st.text_input("Variance","Type Here")
  skewness = st.text_input("skewness","Type Here")
  curtosis = st.text_input("curtosis","Type Here")
  entropy = st.text_input("entropy","Type Here")
  result=""
  if st.button("Predict"):
      result=predict_note_authentication(variance,skewness,curtosis,entropy)
  st.success('The output is {}'.format(result))
  if st.button("About"):
      st.text("Lets LEarn")
      st.text("Built with Streamlit")

if __name__=='__main__':
    main()
    
